In [1]:
import re
import os
import json
import pandas as pd

In [2]:
with open(os.path.join("data_fixes", "opravy.json"), encoding="utf-8") as opravy:
    opravy = json.loads(opravy.read())

In [3]:
persony = pd.read_json(os.path.join("data", "persony_raw.json"))

In [4]:
persony

,Jméno,Id,Medailon,Osoba,Ocenění,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
0,Ondřej Mašek (1000),1000,,fyzická,"[[Sedmero krkavců (400594), Vítěz\n ...",None,None,None,None,None,None
1,kino Světozor (100019),100019,None,právnická,None,None,None,None,None,None,None
2,"AD WOOD, s.r.o. (100038)",100038,None,právnická,None,None,None,None,None,None,None
3,Hypermarket Film s.r.o. (100039),100039,None,právnická,None,Hypermarket Film,None,None,None,None,None
4,Hypermarket Film s.r.o. (100039),100039,None,právnická,None,Hypermarket Film,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...
37417,Josef Dvořák (9998),9998,"herec, divadelní podnikatel",fyzická,"[[None, Vítěz\n ...",Josef Voloďa Dvořák (rodné jméno),None,25.04.1942,"Horní Cerekev, Československo",None,None
37418,Antonín Dimitrov (99996),99996,"výtvarník, grafik, scénograf, designér",fyzická,None,None,None,27.02.1928,"Mšecké Žehrovice, Československo",27.12.2014,"Bobcaygeon, Kanada"
37419,Čechomor (person),person,None,právnická,"[[Rok ďábla (12836), Vítěz\n ...",None,None,None,None,None,None
37420,Kevin Mc Kidd (person),person,,fyzická,None,None,None,None,None,None,None


In [5]:
def odnebovat(x):
    if x:
        if "nebo" in x:
            x = x.split("nebo")[1].strip()
    return x

In [6]:
def rozdel_misto(x):
    if type(x) == str:
        try:
            mesto = x.split(",")[0]
            stat = x.split(",")[1]
        except:
            mesto = None
            stat = x
    else:
        mesto = None
        stat = None
    return [mesto, stat]

## Doplnění dat z Wikidat

In [7]:
persony[persony["Jméno"] == "Stanislav Litera (35280)"].sample(1)

,Jméno,Id,Medailon,Osoba,Ocenění,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
19748,Stanislav Litera (35280),35280,,fyzická,None,None,None,None,None,None,None


In [8]:
with open(os.path.join("data_fixes", "data_z_wikidat.json"), encoding="utf-8") as data_z_wikidat:
    data_z_wikidat = json.loads(data_z_wikidat.read())

In [9]:
for key, val in data_z_wikidat.items():
    persony.loc[persony["Jméno"] == key, "Datum narození"] = val[0]

In [10]:
for key, val in data_z_wikidat.items():
    persony.loc[persony["Jméno"] == key, "Datum úmrtí"] = val[1]

In [11]:
persony[persony["Jméno"] == "Stanislav Litera (35280)"].sample(1)

,Jméno,Id,Medailon,Osoba,Ocenění,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
19748,Stanislav Litera (35280),35280,,fyzická,None,None,None,11.06.1931,None,21.09.2003,None


## Opravy

In [12]:
persony[persony["Jméno"] == "Eva Pecková (34563)"]

,Jméno,Id,Medailon,Osoba,Ocenění,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
19330,Eva Pecková (34563),34563,"pedagožka, zpěvačka, tanečnice",fyzická,None,Eva Kubičková (rodné jméno),None,09.12.1941,"Volyně, Československo",18.07.2018,"České Budějovice, Česká republika"


In [13]:
for key, val in opravy["data_narozeni"].items():
    persony.loc[persony["Jméno"] == key, "Datum narození"] = val

In [14]:
for key, val in opravy["jmena"].items():
    persony.loc[persony["Jméno"] == key, "Jméno"] = val

In [15]:
persony[persony["Jméno"] == "Eva Pecková (34563)"]

,Jméno,Id,Medailon,Osoba,Ocenění,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
19330,Eva Pecková (34563),34563,"pedagožka, zpěvačka, tanečnice",fyzická,None,Eva Kubičková (rodné jméno),None,,"Volyně, Československo",18.07.2018,"České Budějovice, Česká republika"


In [16]:
persony[persony["Jméno"] == "Adam Klemens (1779)"]

,Jméno,Id,Medailon,Osoba,Ocenění,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí
12103,Adam Klemens (1779),1779,"dirigent, hudební skladatel",fyzická,None,None,None,14.01.1967,"Praha, Československo",None,None


## Konverze formátů

In [17]:
persony["Datum narození"].sample(250).unique()

array(['05.08.1910', None, '27.07.1939', '20.05.1974', '18.09.1968',
       '08.06.1948', '14.01.1937', '11.08.1968', '22.02.1919',
       '05.10.1937', '05.11.1943', '19.11.1920', '14.09.1973',
       '01.09.1940', '14.01.1942', '06.01.1965', '18.03.1931',
       '04.02.1857', '24.06.1930', '13.07.1962', '17.10.1902',
       '25.09.1948', '14.09.1891', '30.10.1896', '22.12.1901',
       '20.09.1928', '29.08.1964', '26.08.1995', '19.04.1992',
       '17.09.1912', '16.05.1953', '26.02.1930', '27.12.1992',
       '05.05.1898', '17.09.1907', '14.02.1933', '01.03.1904',
       '2. 6. 1972', '24.07.1906', '28.07.1938', '05.08.1935',
       '20.12.1967', '06.03.1973', '15.05.1911', '07.09.1871',
       '08.01.1947', '22.10.1968', '1887', '1974', '18.05.1973', '1894',
       '1966', '02.12.1943', '10.11.1943', '09.12.1943', '01.06.1920',
       '23.02.1906', '28.03.1890', '12.08.1881', '10.12.1902',
       '03.04.1921', '18.02.1975', '05.11.1920', '17.03.1974',
       '6. 6. 1903', '21.07.196

In [18]:
persony["Město narození"] = persony["Místo narození"].apply(lambda x: rozdel_misto(x)[0])
persony["Stát narození"] = persony["Místo narození"].apply(lambda x: rozdel_misto(x)[1])
persony["Město úmrtí"] = persony["Místo úmrtí"].apply(lambda x: rozdel_misto(x)[0])
persony["Stát úmrtí"] = persony["Místo úmrtí"].apply(lambda x: rozdel_misto(x)[1])
persony["Rok narození"] = pd.to_numeric(persony["Datum narození"].apply(lambda x: x[-4:] if isinstance(x, str) else None), errors="coerce")
persony["Rok úmrtí"] = pd.to_numeric(persony["Datum úmrtí"].str.replace("?","").apply(lambda x: x[-4:] if isinstance(x, str) else None), errors="coerce")

In [19]:
persony[persony['Jméno'].str.contains("Karel Roden")]

,Jméno,Id,Medailon,Osoba,Ocenění,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Město narození,Stát narození,Město úmrtí,Stát úmrtí,Rok narození,Rok úmrtí
22580,Karel Roden nejml. (494),494,herec,fyzická,"[[None, Nominace\n ...",None,None,18.05.1962,"České Budějovice, Československo",None,None,České Budějovice,Československo,None,None,1962.0,NaN
22581,Karel Roden nejml. (494),494,herec,fyzická,"[[None, Nominace\n ...",None,None,18.05.1962,"České Budějovice, Československo",None,None,České Budějovice,Československo,None,None,1962.0,NaN
27386,Karel Roden ml. (73274),73274,"herec, divadelní režisér",fyzická,None,None,None,10.08.1914,"Okříšky, Rakousko-Uhersko",26.08.1992,"České Budějovice, Československo",Okříšky,Rakousko-Uhersko,České Budějovice,Československo,1914.0,1992.0
27387,Karel Roden ml. (73274),73274,"herec, divadelní režisér",fyzická,None,None,None,10.08.1914,"Okříšky, Rakousko-Uhersko",26.08.1992,"České Budějovice, Československo",Okříšky,Rakousko-Uhersko,České Budějovice,Československo,1914.0,1992.0
36503,Karel Roden st. (94209),94209,"herec, divadelní režisér",fyzická,None,Karel Jan Hromada (rodné jméno),None,01.11.1891,"Praha, Rakousko-Uhersko",02.08.1961,"Liberec, Československo",Praha,Rakousko-Uhersko,Liberec,Československo,1891.0,1961.0
36504,Karel Roden st. (94209),94209,"herec, divadelní režisér",fyzická,None,Karel Jan Hromada (rodné jméno),None,01.11.1891,"Praha, Rakousko-Uhersko",02.08.1961,"Liberec, Československo",Praha,Rakousko-Uhersko,Liberec,Československo,1891.0,1961.0


In [20]:
persony[persony['Jméno'].str.contains("Karel Roden")]['Jméno'].drop_duplicates()

22580    Karel Roden nejml. (494)
27386     Karel Roden ml. (73274)
36503     Karel Roden st. (94209)
Name: Jméno, dtype: object

In [21]:
def na_data(sloupec):
    global persony
    persony["Pracovni"] = persony[sloupec]
    persony["Pracovni"] = persony["Pracovni"].apply(lambda x: odnebovat(x))
    for index, row in persony.iterrows():
        try:
            date = pd.to_datetime(row["Pracovni"], format = "%d.%m.%Y", errors="ignore")
        except ValueError:
            date = pd.to_datetime(row["Pracovni"], format = "%d. %m. %Y", errors="ignore")
        except ValueError:
            date = pd.to_datetime(row["Pracovni"], format = "%Y", errors="ignore")
        except ValueError:
            date = None
        persony.at[index, sloupec] = date
    persony = persony.drop(["Pracovni"], axis = 1)

In [22]:
na_data("Datum narození")
na_data("Datum úmrtí")

In [23]:
persony["Rok narození"] = pd.to_numeric(persony["Rok narození"], errors="coerce")
persony["Rok úmrtí"] = pd.to_numeric(persony["Rok úmrtí"], errors="coerce")

In [24]:
persony = persony.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [25]:
persony = persony.drop_duplicates(subset=["Jméno"])

In [26]:
persony

,Jméno,Id,Medailon,Osoba,Ocenění,Alternativní jména,Životopis,Datum narození,Místo narození,Datum úmrtí,Místo úmrtí,Město narození,Stát narození,Město úmrtí,Stát úmrtí,Rok narození,Rok úmrtí
0,Ondřej Mašek (1000),1000,,fyzická,"[[Sedmero krkavců (400594), Vítěz\n ...",None,None,None,None,None,None,None,None,None,None,NaN,NaN
1,kino Světozor (100019),100019,None,právnická,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN
2,"AD WOOD, s.r.o. (100038)",100038,None,právnická,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN
3,Hypermarket Film s.r.o. (100039),100039,None,právnická,None,Hypermarket Film,None,None,None,None,None,None,None,None,None,NaN,NaN
6,Miloš Novotný (100040),100040,,fyzická,"[[None, Vítěz\n ...",None,None,None,None,None,None,None,None,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37417,Josef Dvořák (9998),9998,"herec, divadelní podnikatel",fyzická,"[[None, Vítěz\n ...",Josef Voloďa Dvořák (rodné jméno),None,1942-04-25 00:00:00,"Horní Cerekev, Československo",None,None,Horní Cerekev,Československo,None,None,1942.0,NaN
37418,Antonín Dimitrov (99996),99996,"výtvarník, grafik, scénograf, designér",fyzická,None,None,None,1928-02-27 00:00:00,"Mšecké Žehrovice, Československo",2014-12-27 00:00:00,"Bobcaygeon, Kanada",Mšecké Žehrovice,Československo,Bobcaygeon,Kanada,1928.0,2014.0
37419,Čechomor (person),person,None,právnická,"[[Rok ďábla (12836), Vítěz\n ...",None,None,None,None,None,None,None,None,None,None,NaN,NaN
37420,Kevin Mc Kidd (person),person,,fyzická,None,None,None,None,None,None,None,None,None,None,None,NaN,NaN


## Split na ocenění

In [27]:
oceneni = persony[["Jméno","Ocenění"]]

In [28]:
oceneni.to_json(os.path.join("data", "persony_oceneni.json"), orient="records", date_format="iso")

In [29]:
persony = persony.drop(columns="Ocenění")

In [30]:
persony.to_json(os.path.join("data", "persony.json"), orient="records", date_format="iso")